# Computing Group-Wise L1 Regularization Strengths from SHAP Values 
# L1-Grouped-Epigenetics

Compute group-wise L1 regularization strengths for epigenetic feature groups based on SHAP values. The process includes SHAP extraction, contribution analysis, normalization and the final computation of the strengths using a 5-step procedure.

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import pearsonr

## Step 1: Extract SHAP Values and Split into Feature Groups

Compute the absolute mean SHAP values and organize them into their respective groups (features, epigenetic codes, etc.).

In [2]:
shap_values = np.load("path/to/shap_values.npy")
print(f"SHAP Values Shape: {shap_values.shape}")

SHAP Values Shape: (1, 50, 264)


In [3]:
shap_values = shap_values.squeeze()
print(f"Squeezed SHAP shape: {shap_values.shape}")

Squeezed SHAP shape: (50, 264)


In [4]:
mean_shap_values = np.mean(np.abs(shap_values), axis=0)

In [5]:
shap_df = pd.DataFrame({
    "Feature": [f"Feature {i+1}" for i in range(mean_shap_values.shape[0])],
    "Mean_SHAP_Value": mean_shap_values
})
# shap_df

In [6]:
f_length=24
group_names = ["features", "feature_ont", "feature_offt", "on_epigenetic_code", "off_epigenetic_code"]
group_ranges = [0, f_length, 2*f_length, 3*f_length, 7*f_length, len(shap_df)]

In [7]:
# Split the SHAP values into named groups
feature_groups = {
    name: shap_df.iloc[start:end]
    for name, start, end in zip(group_names, group_ranges[:-1], group_ranges[1:])
}

In [8]:
features = feature_groups['features']
feature_ont = feature_groups['feature_ont']
feature_offt = feature_groups['feature_offt']
on_epigenetic_code = feature_groups['on_epigenetic_code']
off_epigenetic_code = feature_groups['off_epigenetic_code']

In [9]:
# 24 features for each epigenetic factor
factors = ["CTCF", "DNase", "H3K4me3", "RRBS"]
features_per_factor = 24
num_of_factors = 4

In [10]:
on_target_factors = {
    f"On-Target - {factors[i]}": on_epigenetic_code.iloc[i * features_per_factor:(i + 1) * features_per_factor]
    for i in range(num_of_factors)
}

In [11]:
off_target_factors = {
    f"Off-Target - {factors[i]}": off_epigenetic_code.iloc[i * features_per_factor:(i + 1) * features_per_factor]
    for i in range(num_of_factors)
}

In [12]:
data = {**on_target_factors, **off_target_factors}

# for factor_name, factor_data in data.items():
#     print(f"\n{factor_name}:\n")
#     print(factor_data.head())

## Step 2: Compute the Absolute Total Mean Contribution and Correlation

The absolute total mean contribution and the correlation between the target and off-target features for each of the epigenetic factors (CTCF, DNase, etc.) was calculated based on the SHAP values.

In [13]:
factor_mean_contribution = {
    factor: group['Mean_SHAP_Value'].mean()
    for factor, group in data.items()
}

In [14]:
on_target_contributions = {key: contribution for key, contribution in factor_mean_contribution.items() if "On-Target" in key}
off_target_contributions = {key: contribution for key, contribution in factor_mean_contribution.items() if "Off-Target" in key}

In [15]:
print("Mean Contribution of On-Target Factors:")
for name, value in on_target_contributions.items():
    print(f"{name:<20} {value:.8f}")

Mean Contribution of On-Target Factors:
On-Target - CTCF     0.00039243
On-Target - DNase    0.00056323
On-Target - H3K4me3  0.00044109
On-Target - RRBS     0.00046129


In [16]:
print("Mean Contribution of Off-Target Factors:")
for name, value in off_target_contributions.items():
    print(f"{name:<20} {value:.8f}")

Mean Contribution of Off-Target Factors:
Off-Target - CTCF    0.00059545
Off-Target - DNase   0.00068344
Off-Target - H3K4me3 0.00072015
Off-Target - RRBS    0.00073305


### Total Mean Contribution (On-Target and Off-Target) for Each Factor

In [17]:
total_mean_contributions = {
    factor: (on_target_contributions[f"On-Target - {factor}"] + off_target_contributions[f"Off-Target - {factor}"]) / 2
    for factor in factors
}

print("Mean Contribution of Factors:")
for name, value in total_mean_contributions.items():
    print(f"{name:<10} {value:.8f}")

Mean Contribution of Factors:
CTCF       0.00049394
DNase      0.00062333
H3K4me3    0.00058062
RRBS       0.00059717


### Pearson Correlation Between On-Target and Off-Target for Each Factor

In [18]:
correlations = {}

for i, factor in enumerate(factors):
    on_target_values = on_target_factors[f"On-Target - {factor}"]['Mean_SHAP_Value']
    off_target_values = off_target_factors[f"Off-Target - {factor}"]['Mean_SHAP_Value']

    corr, _ = pearsonr(on_target_values, off_target_values)
    correlations[factor] = corr

In [19]:
print("Correlation between Target and Off-Target for each epigenetic factor:")
for name, value in correlations.items():
    print(f"{name:<10} {value:.8f}")

Correlation between Target and Off-Target for each epigenetic factor:
CTCF       0.50313665
DNase      0.22661480
H3K4me3    0.43399773
RRBS       0.29125054


## Step 3: Normalize the Total Mean Contribution and the Correlation Values
#### x_norm = (x - min) / (max - min)

In [20]:
normalized_mean_contributions = {}
min_contribution = min(total_mean_contributions.values())
max_contribution = max(total_mean_contributions.values())

for factor, value in total_mean_contributions.items():
    normalized_mean_contributions[factor] = (value - min_contribution) / (max_contribution - min_contribution)
    
print("Normalized Mean SHAP Contributions:")
for name, value in normalized_mean_contributions.items():
    print(f"{name:<10} {value:.8f}")

Normalized Mean SHAP Contributions:
CTCF       0.00000000
DNase      1.00000000
H3K4me3    0.66987653
RRBS       0.79778294


In [21]:
normalized_correlations = {}
min_corr = min(correlations.values())
max_corr = max(correlations.values())

for factor, value in correlations.items():
    normalized_correlations[factor] = (value - min_corr) / (max_corr - min_corr)

print("\nNormalized Correlations:")
for name, value in normalized_correlations.items():
    print(f"{name:<10} {value:.8f}")


Normalized Correlations:
CTCF       1.00000000
DNase      0.00000000
H3K4me3    0.74996944
RRBS       0.23374551


## Step 4: Compute the Weight of Each Factor
#### w(i) = α · contribution(i) + (1 - α) · correlation(i)

In [22]:
alpha = 0.75
weights = {}

for factor in normalized_mean_contributions.keys():
    weights[factor] = (
        alpha * normalized_mean_contributions[factor] + (1 - alpha) * normalized_correlations[factor]
    )
    
print("Computed Weights for Each Epigenetic Factor:")
for name, value in weights.items():
    print(f"{name:<10} {value:.8f}")

Computed Weights for Each Epigenetic Factor:
CTCF       0.25000000
DNase      0.75000000
H3K4me3    0.68989975
RRBS       0.65677358


## Step 5: Compute the Regularization Factors’ Strengths
#### λ(i) = k / (w(i) + ε), where ε = 1e-6

In [23]:
k=0.01
epsilon = 1e-6
lambda_g = {}

for factor, weights in weights.items():
    lambda_g[factor] = round(k/(weights + epsilon),8)
    
print("Regularization Strengths per Factor:")
for name, value in lambda_g.items():
    print(f"{name:<10} {value:.8f}")

Regularization Strengths per Factor:
CTCF       0.03999984
DNase      0.01333332
H3K4me3    0.01449484
RRBS       0.01522592
